# Data Modeling
Zuerst vorbereitete Daten einlesen

In [14]:
import pandas as pd;
import numpy as np;
from sklearn.model_selection import train_test_split
from sklearn import tree

cleaned_data = pd.read_csv('cleaned_data.csv')

C:\Users\I586521\AppData\Local\Temp\ipykernel_26208\257053229.py:6: DtypeWarning: Columns (9,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_data = pd.read_csv('cleaned_data.csv')


### Robert 
--Predict the category of crime that occurred given a certain time and location

### Daten Normalisieren 
Einige Datumsspalten werden als Cosinus-Funktion ausgegeben.
Grund: Damit 24Uhr und 1 Uhr auch für das Modell nebeneinander sind. 

Es wird bei Stunden, Tagen und Monaten gemacht. 

In [15]:
# Erstellen der neuen Spalte in "cleaned_data"
cleaned_data['TIME.OCC_hour_cos'] = np.cos(2 * np.pi * cleaned_data['TIME.OCC_hour'] / 24)
cleaned_data['DATE.OCC_day_cos'] = np.cos(2 * np.pi * cleaned_data['DATE.OCC_day'] / 31)
#cleaned_data['DATE.OCC_weekday_cos'] = np.cos(2 * np.pi * cleaned_data['TIME.OCC_weekday'] / 7) # nicht möglich da keine Zahlenwerte
cleaned_data['DATE.OCC_month_cos'] = np.cos(2 * np.pi * cleaned_data['DATE.OCC_month'] / 12)


#### Location aufteilen in Dummy Varibalen 


In [16]:
# Spalte LOCATION wird zu mehrere 0,1 Variablen um diese im Suchbaum abzubilden 

# One-Hot Encoding für das String-Feature "LOCATION"
area_names = list(map(lambda x: 'AREA.NAME_' + x, set(cleaned_data['AREA.NAME'].values)))
encoded_data = pd.get_dummies(cleaned_data, columns=['AREA.NAME'])
cleaned_data = pd.concat([encoded_data], axis=1)
# Anzeigen des transformierten DataFrames

In [17]:
cleaned_data.head()

,Date.Rptd,DR.NO,DATE.OCC,TIME.OCC,AREA,RD,Crm.Cd,CrmCd.Desc,Status,Status.Desc,...,AREA.NAME_Olympic,AREA.NAME_Pacific,AREA.NAME_Rampart,AREA.NAME_Southeast,AREA.NAME_Southwest,AREA.NAME_Topanga,AREA.NAME_Van Nuys,AREA.NAME_West LA,AREA.NAME_West Valley,AREA.NAME_Wilshire
0,2013-03-20,132007717.0,2013-03-20,20:15:00,20,2004,997,TRAFFIC DR #,UNK,Unknown,...,True,False,False,False,False,False,False,False,False,False
1,2013-03-10,130608787.0,2013-03-10,04:45:00,6,635,997,TRAFFIC DR #,UNK,Unknown,...,False,False,False,False,False,False,False,False,False,False
2,2013-12-18,131820260.0,2013-12-18,07:45:00,18,1839,997,TRAFFIC DR #,UNK,Unknown,...,False,False,False,True,False,False,False,False,False,False
3,2013-10-18,131817514.0,2013-10-18,17:30:00,18,1827,997,TRAFFIC DR #,UNK,Unknown,...,False,False,False,True,False,False,False,False,False,False
4,2013-05-26,130510483.0,2013-05-25,20:00:00,5,507,440,THEFT PLAIN - PETTY (UNDER $400),UNK,Unknown,...,False,False,False,False,False,False,False,False,False,False


In [18]:
dec_tree = tree.DecisionTreeClassifier()
features = ['UNIX.TIMESTAMP','LAT','LONG', 'RD'] + area_names
target = 'CRIME_CAT'

selection = features + [target]

predict_data = cleaned_data[selection].copy()

X_train, X_test, y_train, y_test = train_test_split(predict_data.drop(target, axis=1), predict_data[target], test_size=0.2, random_state=42)
dec_tree.fit(X_train,y_train)

print("Model Accuracy:")
dec_tree.score(X_test,y_test)

Model Accuracy:


0.41317263188400855

#### DecisionTree
- Target: CRIME_VIOLENT

In [19]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
#import graphviz
from sklearn.tree import export_graphviz

predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['AREA', 'RD']
target = 'CRIME_VIOLENT'

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: LabelEncoder auf 'AREA' anwenden
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

# Schritt 5: LabelEncoder auf 'RD' anwenden
label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 8)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

Genauigkeit des Modells: 0.7400165361109454


#### DecisionTree
- Target: CRIME_CAT
- Features: ['AREA', 'RD',  'LAT', 'LONG']

In [20]:
predict_data = cleaned_data.copy()

# Schritt 2: Merkmale und Zielvariable definieren
features = ['AREA', 'RD',  'LAT', 'LONG']
target = 'CRIME_CAT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: LabelEncoder auf 'AREA' anwenden
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

# Schritt 5: LabelEncoder auf 'RD' anwenden
label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

# Schritt 4: Modell erstellen und trainieren
model = DecisionTreeClassifier(max_depth = 8)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells:", accuracy)

Genauigkeit des Modells: 0.4247606951985612


#### DecisionForest 1/3
- Target: CRIME_CAT
- Fetures:['AREA', 'RD', 'LAT', 'LONG']

Bisher bestes Ergebnis!

In [21]:
#Forrest

predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
features = ['AREA', 'RD', 'LAT', 'LONG']
target = 'CRIME_CAT'

# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: LabelEncoder auf 'AREA' anwenden
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

# Schritt 5: LabelEncoder auf 'RD' anwenden
label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

# Schritt 4: Modell erstellen und trainieren

model = RandomForestClassifier(max_depth = 21, n_estimators=10)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells deep: " , accuracy)
#deep = 16, nest = 10 : 46,6%
#deep = 16 nest = 100 : 46,9
#deep = 20 nest = 10: 47,62%
#deep = 32 nest = 10 : 46,8 - ab n= 32 wird es nicht besser
#iteration von 16-32 durchgeführt. Beste Genauigkeit bon deep = 21: 48,63%

Genauigkeit des Modells deep:  0.49606201893981366


#### DecisionForest 2/3
- Target: CRIME_CAT
- Features: alle außer: 

In [22]:
spaltennamen = list(cleaned_data.columns)
print(spaltennamen)

['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA', 'RD', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LAT', 'LONG', 'LOCATION.house_number', 'LOCATION.street', 'Cross.Street.house_number', 'Cross.Street.street', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_weekend', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT', 'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos', 'AREA.NAME_77th Street', 'AREA.NAME_Central', 'AREA.NAME_Devonshire', 'AREA.NAME_Foothill', 'AREA.NAME_Harbor', 'AREA.NAME_Hollenbeck', 'AREA.NAME_Hollywood', 'AREA.NAME_Mission', 'AREA.NAME_N Hollywood', 'AREA.NAME_Newton', 'AREA.NAME_Northeast', 'AREA.NAME_Olympic', 'AREA.NAME_Pacific', 'AREA.NAME_Rampart', 'AREA.NAME_Southeast', 'AREA.NAME_Southwest', 'AREA.NAME_Topanga', 'AREA.NAME_Van Nuys', 'AREA.NAME_West LA', 'AREA.NAME_West Valley', 'AREA.NAME_Wilshire']


In [24]:
#Forrest

predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
#features = ['AREA', 'RD', 'LAT', 'LONG']
target = 'CRIME_CAT'
#column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA', 'AREA.NAME', 'RD', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'LAT', 'LONG', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT', 'Cos_Uhrzeit', 'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos']
column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION.street', 
                     'Cross.Street.street', 'LOCATION.house_number', 'Cross.Street.house_number','UNIX.TIMESTAMP', 'TIME.OCC_hour', 
                     'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT']
#Enthalten: 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos'
features = predict_data.drop([target] + column_to_exclude, axis=1).columns
# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: LabelEncoder auf 'AREA' anwenden
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

# Schritt 5: LabelEncoder auf 'RD' anwenden
label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

maxd = 800
nest = 100

model = RandomForestClassifier(max_depth = maxd, n_estimators=nest)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells deep: " + str(maxd) + ", " + "n_est: " + str(nest) + ": " +  str(accuracy))

#iteration von 16-32 durchgeführt. Beste Genauigkeit bon deep = 21: 48,63%
#Genauigkeit des Modells deep: 16, 10: 0.4599616398945097
#Genauigkeit des Modells deep: 30, n_est: 100: 0.44278838896436595
#Genauigkeit des Modells deep: 21, n_est: 100: 0.47115444382286864
#Genauigkeit des Modells deep: 20, n_est: 100: 0.47222000230871003
#Genauigkeit des Modells deep: 20, n_est: 40: 0.47016880222346535
#Genauigkeit des Modells deep: 19, n_est: 100: 0.47237983608158624

#['ASSAULT_BATTERY_AND_ROBBERY', 'CHILD_ABUSE', 'CHILD_NEGLECT', 'DISTURBANE_OF_PUBLIC_ORDER', 'FIREARMS_AND_EXPLOSIVES_OFFENSES', 'FRAUD_AND_COUNTERFEIT', 'HARASSMENT', 'HOMICIDE', 'KIDNAPPING', 'OTHER_NON_VIOLENT_CRIME', 'POLICE_CRIMES', 'PROPERTY_CRIMES', 'SEXUAL_CRIMES', 'SEXUAL_OFFENSES', 'SPECIAL_NON_VIOLENT_CRIME', 'SPECIAL_VIOLENT_CRIMES', 'THREAT_OFFENSES', 'VIOLATION_OF_POLICE_OR_COURT_ORDER']


#Genauigkeit des Modells deep: 80, n_est: 20: 0.43948071783123327
#Genauigkeit des Modells deep: 10, n_est: 20: 0.4376248701350595

Genauigkeit des Modells deep: 800, n_est: 100: 0.45356677094076836


In [25]:
from sklearn import tree
import pydotplus

class_names = [str(c) for c in model.classes_]

# Exportieren des ersten Entscheidungsbaums aus dem Random Forest-Modell
estimator = model.estimators_[0]

# Erstellen des Dot-Datenstrings
dot_data = tree.export_graphviz(estimator, out_file=None, feature_names=features, class_names=class_names, filled=True, rounded=True, special_characters=True)

# Erzeugen des Graphobjekts aus dem Dot-Datenstring
graph = pydotplus.graph_from_dot_data(dot_data)

# Speichern des Entscheidungsbaums als PNG-Bild
graph.write_png("randomForest.png")

# Speichern des Entscheidungsbaums als PDF
graph.write_pdf("randomForest.pdf")


KeyboardInterrupt: 

#### DecisionForest 3/3
- Target: Crm.Cd
- Features: alle außer: 

In [26]:
predict_data = cleaned_data.copy()
from sklearn.ensemble import RandomForestClassifier
# Schritt 2: Merkmale und Zielvariable definieren
#features = ['AREA', 'RD', 'LAT', 'LONG']
target = 'Crm.Cd'
#column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA', 'AREA.NAME', 'RD', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'LAT', 'LONG', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT', 'Cos_Uhrzeit', 'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos']
column_to_exclude = ['Date.Rptd', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION.street', 
                     'Cross.Street.street', 'LOCATION.house_number', 'Cross.Street.house_number','UNIX.TIMESTAMP', 'TIME.OCC_hour', 
                     'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT']
#Enthalten: 'AREA', 'RD', 'LAT', 'LONG',  'TIME.OCC_hour_cos', 'DATE.OCC_day_cos', 'DATE.OCC_month_cos'
features = predict_data.drop([target] + column_to_exclude, axis=1).columns
# Remove all NaN values
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()


# Schritt 3: Datensatz in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 4: LabelEncoder auf 'AREA' anwenden
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

# Schritt 5: LabelEncoder auf 'RD' anwenden
label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

maxd = 21
nest = 10

model = RandomForestClassifier(max_depth = maxd, n_estimators=nest)
model.fit(X_train, y_train)

# Schritt 5: Vorhersagen treffen
y_pred = model.predict(X_test)

# Schritt 6: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells deep: " + str(maxd) + ", " + "n_est: " + str(nest) + ": " +  str(accuracy))

#iteration von 16-32 durchgeführt. Beste Genauigkeit bon deep = 21: 48,63%
#Genauigkeit des Modells deep: 16, 10: 0.4599616398945097
#Genauigkeit des Modells deep: 30, n_est: 100: 0.44278838896436595
#Genauigkeit des Modells deep: 21, n_est: 100: 0.47115444382286864
#Genauigkeit des Modells deep: 20, n_est: 100: 0.47222000230871003
#Genauigkeit des Modells deep: 20, n_est: 40: 0.47016880222346535
#Genauigkeit des Modells deep: 19, n_est: 100: 0.47237983608158624

#['ASSAULT_BATTERY_AND_ROBBERY', 'CHILD_ABUSE', 'CHILD_NEGLECT', 'DISTURBANE_OF_PUBLIC_ORDER', 'FIREARMS_AND_EXPLOSIVES_OFFENSES', 'FRAUD_AND_COUNTERFEIT', 'HARASSMENT', 'HOMICIDE', 'KIDNAPPING', 'OTHER_NON_VIOLENT_CRIME', 'POLICE_CRIMES', 'PROPERTY_CRIMES', 'SEXUAL_CRIMES', 'SEXUAL_OFFENSES', 'SPECIAL_NON_VIOLENT_CRIME', 'SPECIAL_VIOLENT_CRIMES', 'THREAT_OFFENSES', 'VIOLATION_OF_POLICE_OR_COURT_ORDER']


#Genauigkeit des Modells deep: 80, n_est: 20: 0.43948071783123327
#Genauigkeit des Modells deep: 10, n_est: 20: 0.4376248701350595

KeyError: "['AREA.NAME', 'LOCATION', 'Cross.Street', 'Cos_Uhrzeit'] not found in axis"

####  sequentielles neuronales Netzwerk verwendet.
- Dense-Schicht

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Schritt 1: Daten laden und vorverarbeiten
predict_data = cleaned_data.copy()
target = 'CRIME_CAT'
column_to_exclude = ['Date.Rptd', 'Cos_Uhrzeit', 'DR.NO', 'DATE.OCC', 'TIME.OCC', 'AREA.NAME', 'Crm.Cd', 'CrmCd.Desc', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'LAT', 'LONG', 'UNIX.TIMESTAMP', 'TIME.OCC_hour', 'DATE.OCC_day', 'DATE.OCC_weekday', 'DATE.OCC_month', 'DATE.OCC_year', 'CRIME_VIOLENT', 'CRIME_CAT']
#column_to_exclude = ['Crm.Cd', 'Cross.Street', 'Date.Rptd', 'DATE.OCC', 'TIME.OCC', 'DR.NO', 'AREA.NAME',  'CrmCd.Desc', 'Status', 'Status.Desc',  'LOCATION',   'TIME.OCC_hour','CRIME_VIOLENT','DATE.OCC_weekday',  'CRIME_CAT']
features = predict_data.drop([target] + column_to_exclude, axis=1).columns

# Entferne alle NaN-Werte
predict_data = predict_data[predict_data['LAT'].notnull() & predict_data['LONG'].notnull()].copy()

# Schritt 2: Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(predict_data[features], predict_data[target], test_size=0.2, random_state=42)

# Schritt 3: Merkmale kodieren
label_encoder_area = LabelEncoder()
X_train['AREA'] = label_encoder_area.fit_transform(X_train['AREA'])
X_test['AREA'] = label_encoder_area.transform(X_test['AREA'])

label_encoder_rd = LabelEncoder()
all_rd_labels = pd.concat([X_train['RD'], X_test['RD']])
label_encoder_rd.fit(all_rd_labels)
X_train['RD'] = label_encoder_rd.transform(X_train['RD'])
X_test['RD'] = label_encoder_rd.transform(X_test['RD'])

# Schritt 4: Datentypen konvertieren
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = label_encoder_area.fit_transform(y_train)
y_test = label_encoder_area.transform(y_test)

# Schritt 5: Neuronales Netzwerk erstellen

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, activation='tanh', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(1, activation='tanh'))
model.add(tf.keras.layers.Dense(label_encoder_area.classes_.shape[0], activation='softmax'))

# Schritt 6: Modell kompilieren und trainieren
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size=20, validation_split=0.2)

# Schritt 7: Vorhersagen treffen
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Schritt 8: Modell evaluieren
accuracy = accuracy_score(y_test, y_pred)
print("Genauigkeit des Modells: " + str(accuracy))

#höher als 42,98 ist nicht möglich. Habe activation = "tanh" und "relu" ausprobiert, 2 und 3 Layers und 32, 64 und 128 Neuronen. 1, 2, 10 Epochs und batchsize = 2, 10, 20 
#"mean_squared_error" funktioniert garnicht
#optimizer='adam' gut, optimizer='SGD' gut,  optimizer='Adagrad' gut
# okay. iwas ist komisch. mit 3 mal 2 Neuronen bekomme ich trd über 42%. Das ist komisch.



Epoch 1/2
36038/36038 [==============================] - 38s 1ms/step - loss: 1.5877 - accuracy: 0.4224 - val_loss: 1.5654 - val_accuracy: 0.4298
Epoch 2/2
7039/7039 [==============================] - 5s 639us/step
Genauigkeit des Modells: 0.4281147606489251


#### Naive Bayes-Klassifikator (GaussianNB)
- Target: CRIME_CAT
- Features: ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB


predict_data = cleaned_data.copy()
# Build a Gaussian Classifier
nb_class = GaussianNB()
# Prior-Verteilungen für jede Klasse definieren

features = ['UNIX.TIMESTAMP','AREA.NAME','LAT','LONG']
target = 'CRIME_CAT'
selection = features + [target]

predict_data = cleaned_data[selection].copy()
# Filtern von null (muss nur für eines gemacht werden, da Koordinaten nie alleine vorkommen)
predict_data = predict_data[predict_data['LONG'].notnull()]
predict_data_encoded = pd.get_dummies(predict_data, columns=['AREA.NAME'])

X_train, X_test, y_train, y_test = train_test_split(predict_data_encoded.drop(target, axis=1), predict_data_encoded[target], test_size=0.2, random_state=42)
nb_class.fit(X_train,y_train)

print("Model Accuracy:")
nb_class.score(X_test,y_test)
#ohne Priors: 42,811
#probiert Priors anzupassen. Komme auf Maximal 21% WK. lohnt nicht
#priors = [0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0555, 0.0565] - 8,9%

#Auch weitere Anpassungen brachten keine Verbesserung:
#print("Accuracy (Standardized):", accuracy_standardized)
#print("Accuracy (Normalized):", accuracy_normalized)

Model Accuracy:


0.4281147606489251